In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive
!pip install imageio

In [0]:
import tensorflow as tf
import tensorflow.contrib.keras.api.keras.backend as K

from tensorflow.contrib.keras.api.keras.optimizers import Adam
from tensorflow.contrib.keras.api.keras.models import Model, Sequential
from tensorflow.contrib.keras.api.keras.layers import Input , Activation
from tensorflow.contrib.keras.api.keras.layers import Conv2D, Reshape
from tensorflow.contrib.keras.api.keras.layers import Dropout,BatchNormalization
from tensorflow.contrib.keras.api.keras.layers import concatenate, Conv2DTranspose

import numpy as np
import os
import random
import matplotlib
import imageio
#from model import network_stereo_basic_short as network_v1

from scipy.misc import imsave
import matplotlib.pyplot as plt

In [0]:
def show_flow(tmp):
    ang=np.arctan2(tmp[:,:,1],tmp[:,:,0])
    ang=ang-ang.min()
    ang=ang/ang.max()
    mag=(tmp[:,:,0]**2+tmp[:,:,1]**2)**0.5

    image1=np.ones((tmp.shape[0], tmp.shape[1], 3))
    image1[:,:,0]=ang
    image1[:,:,2]=mag/np.max(mag)
    image1=matplotlib.colors.hsv_to_rgb(image1)
    return image1


In [0]:
""" 
// Step1-1 train/validation/test 데이터 로드. // 

img_L: 0 ~ 255
img_R: 0 ~ 255
disp_L: -15 ~ 15 (px)
disp_R: -15 ~ 15 (px)
"""
print('data loading...', end ='  ')
im1=1/255*np.load('drive/colab/data_flow/flow1.npy')
im2=1/255*np.load('drive/colab/data_flow/flow2.npy')
flow_map=np.load('drive/colab/data_flow/flowGT.npy')



im1_test=1/255*np.load('drive/colab/data_flow/flow1_test.npy')
im2_test=1/255*np.load('drive/colab/data_flow/flow2_test.npy')
flow_map_test=np.load('drive/colab/data_flow/flowGT_test.npy')
im1_test=np.transpose(im1_test,[3,0,1,2])[:,:-5]
im2_test=np.transpose(im2_test,[3,0,1,2])[:,:-5]
flow_map_test=np.transpose(flow_map_test,[3,0,1,2])[:,:-5]
# 데이터 shape 받아오기 (image_h, image_w, 3, )
image_h=im1_test.shape[0] # 109
image_w=im1_test.shape[1] # 256    

plt.subplot(311); plt.axis('off')
plt.imshow(im1[:,:,:,7])
plt.subplot(312); plt.axis('off')
plt.imshow(im2[:,:,:,7])
plt.subplot(313); plt.axis('off')
plt.imshow(show_flow(flow_map[:,:,:,7]))
print('done')

In [0]:
""" 
// Step1-2 기타 setting  // 

conv_depth: Conv2D 레이어 개수
filt_num: Conv2D 필터 개수

patch_h, patch_w: 트레이닝할 때 사용할 patch 크기
batch_size, learning_rate
one_step, epoches
"""

network_name="flow_test1"
save_bool = False
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"  

conv_depth=8
filt_num=64

batch_size=16
learning_rate=1e-4
decay_rate=1e-6

patch_h=4*conv_depth+8
patch_w=4*conv_depth+8

one_step=3000 
epoches=20
iter00=0  

In [0]:
"""// Step2 네트워크 설계 // """
def network_flow(patch_h,patch_w,conv_depth,filt_num):
    """
    네트워크 설계
    < Layer 사용 예시 >
    output=concatenate([input1,input2])
    input1=Conv2D(filters, kernel_size, strides=(1, 1), padding='valid', 
                  dilation_rate=(1, 1))(input1)
    input1=BatchNormalization()(input1)
    input1=Activation('relu')(input1)
    
    """
    
    ''' Define Input_L, input_R  ''' 
    input_im1 = Input(shape=(patch_h,patch_w,3), name='input_im1')
    input_im2= Input(shape=(patch_h,patch_w,3), name='input_im2')
    
    
    """
    write your code
    im_stack = ?
    
    """
    
    model = Model(inputs = [input_im1,input_im2], outputs = [im_stack])
    
    return model  

In [0]:
    """ 
    
    
    training에서 사용할 모델 정의 """ 
    model_train=network_flow(patch_h,patch_w,conv_depth,filt_num)
    opt = Adam(lr=learning_rate,decay=decay_rate)
    model_train.compile(optimizer=opt, loss='mse')#mse_valid)
    model_train.summary() 
    
    
    """ input/output size가 다를때 크기 맞추기 위해 crop 해줌  """
    sliced_half=(patch_w-model_train.output_shape[1])//2


    """ val/test에서 사용할 모델 정의  """
    
    model_val=network_flow(image_h,image_w,conv_depth,filt_num)
    model_val.compile(optimizer=opt, loss='mse')

In [0]:
""" 
Step3 트레이닝 데이터 생성
data 생성함수 정의 (fit_generator에서 사용할 것임)
FCN 이므로 patch-wise training으로 학습하는 것이 학습속도 빠름.
patch 크기: (patch_h, patch_w)
이미지 랜덤으로 잘라서 batch_img_L: (batch_size,patch_h, patch_w) 만든다.

""" 
def dataGenerator(input_im1,input_im2,flow_map, batch_size, patch_h, patch_w, sliced_half): 
    while 1:
        """
        실시간으로 트레이닝 데이터 생성하는 함수 (fit_generator에서 사용할 예정)
        이미지 사이즈에 맞춰 랜덤으로 위치를 바꿔가며 batch_size만큼 생성한다.
        Data augmentation(color 등)도 해보기.
        
        <Input>등
        im1 : (109, 256, 3, N)  N장의 이전프레임
        im2 : (109, 256, 3, N)  N장의 현재프레임
        flow_map : (109, 256, 2, N) N장의 flow image
        batch_size,                배치 크기
        patch_h, patch_w           crop할 크기(패치 크기)
        sliced_half                zero-padding 없을 때, 이미지가 작아지는 값의 1/2
        
        <Output> 
        batch_img_L: (batch_size, patch_h, patch_w, 3)
        batch_img_R: (batch_size, patch_h, patch_w, 3)
        batch_disp_L: (batch_size, patch_h, int(patch_w-2*sliced_half))
        """
        
        batch_im1 = np.zeros((batch_size, patch_h, patch_w, 3))
        batch_im2 = np.zeros((batch_size, patch_h, patch_w, 3))
        batch_flow_map= np.zeros((batch_size, int(patch_w-2*sliced_half), int(patch_w-2*sliced_half), 2))
        
        for _ in range(batch_size):
            image_id=int(im1.shape[-1]*random.random())
            idx=int((im1.shape[0]-patch_h)*random.random())
            idy=int((im1.shape[1]-patch_w)*random.random())
            rand_scale=0.8+0.199*random.random()
            rand_add=(1-rand_scale)+random.random()
            """
            write your code
            batch_im1 ?
            batch_im2 ?
            batch_flow_map?

            """            


        yield([batch_im1,batch_im2],batch_flow_map)
  
my_generator = dataGenerator(im1, im2, flow_map, 
                             batch_size, patch_h, patch_w, sliced_half)

In [0]:
for i in range((epoches)):


    """ Training:
        3 epoch씩 학습하고, 
        1 epoch는 3000 """
    model_train.fit_generator(my_generator, steps_per_epoch = one_step, 
                        epochs = iter00+3,  initial_epoch=iter00,
                        verbose=1, workers=1)
    iter00=iter00+3



    """ Test: 
        테스트이미지로 중간중간 결과 확인.
    """         
    weight_tmp1=model_train.get_weights() 
    model_val.set_weights(weight_tmp1)


    predict_test=model_val.predict([im1_test, im2_test],batch_size=1)
    predict_test=np.squeeze(predict_test)   


    """ 
    테스트 결과 저장
    ( 이미지->gif, weights->hdf5 )  
    """  

    gif_images = []
    for _ in range(len(predict_test)):
        gif_images.append((np.concatenate([im2_test[_,sliced_half:-sliced_half,
                                                   sliced_half:-sliced_half],
                                          show_flow(predict_test[_,])],axis=0 ) ).astype(np.uint8) )
    imageio.mimsave('save/%s_flow_iter%04d.gif' % (network_name,iter00), 
                    gif_images)
    save_path="drive/colab/save/%s_iter%04d.hdf5" % (network_name,iter00)
    if(save_bool==True):
        model_train.save_weights(save_path)
    print(save_path)